In [ ]:
# STEP 1: Create NY subset of data (NYSubmissions.csv)
    # Find all NY Facilities in the CDFC_FacilityCodes.csv and save as a set
    # Use the set to filter complaint-filings.csv so that we are left with only complaints made at NY facilities 
    # Save results to NYSubmissions.csv

import pandas as pd

# Load facilities data, filter for NY State facility codes, and save as a variable (nyfacilitycodes)
allfacilities = pd.read_csv('../data/CDFC_FacilityCodes.csv')
nyfacilitycodes = set(allfacilities.loc[allfacilities['State'] == 'NY', 'Facility_Code']) # Replace "NY" with another state code, if desired

# Load BOP submissions data and filter where CDFCLEVN value (Facility of Occurrence) is in nyfacilitycodes
allsubmissions = pd.read_csv('../data/complaint-filings.csv')
nysubmissions = allsubmissions[allsubmissions['CDFCLEVN'].isin(nyfacilitycodes)]

# Save filtered NY submissions to a CSV file 
nysubmissions.to_csv('../results/data/NY_Submissions.csv', index=False) # If creating subset of data for different state, consider changing file name to avoid confusion

/var/folders/x3/5gclj9rd6tj1rpgd5cwxqvy00000gn/T/ipykernel_39821/1059803717.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [24]:
# STEP 2: Create a new enriched dataset (NYSubmissionsEnriched) by adding columns
  # Add column (subcount) which counts total number of submissions associated with that Case Number (CASENBR)
  # Add column (rejcount) which counts the total number of "Rejected" (rej) submissions associated with that Case Number (CASENBR)
  # Add column (cldclocount) which counts the total number of "Closed Denied" and "Closed Granted" submissions associated with that Case Number (CASENBR)
  # Add column (clgacccount) which counts the total number of "Accepted" and "Closed Granted" submissions associated with that case number (CASENBR)
  # Add column (earliestsitdtrcv) which shows earliest Remedy Case Submission Date Received (sitdtrcv) for that case number (CASENBR)
  # Add column (latestsdtstat) which shows the most recent Date Latest Status Assigned (sdtstat) for that Case Number (CASENBR)
  # Add column (daysbetween) which shows the number of days between earliestsitdtrcv and latestsdtstat. This is the amount of time either between the first complaint and the latest case status.

# Load NYComplaintFilings.csv dataset
nysubmissions2 = pd.read_csv(r'../results/data/NY_Submissions.csv')

# Add a column that counts the total number of submissions associated with that one CASENBR
subcounts = nysubmissions2['CASENBR'].value_counts()
nysubmissions2['subcount'] = nysubmissions2['CASENBR'].map(subcounts)

# Add a column to count the total number of "Rejected" submissions for each CASENBR
rejcounts = nysubmissions2[nysubmissions2['CDSTATUS'] == 'REJ'].groupby('CASENBR').size()
nysubmissions2['rejcount'] = nysubmissions2['CASENBR'].map(rejcounts).fillna(0).astype(int)

# Add a column to count the total number of "Closed Denied" and "Closed Other" submissions for each CASENBR
cldclocounts = nysubmissions2[nysubmissions2['CDSTATUS'].isin(['CLD', 'CLO'])].groupby('CASENBR').size()
nysubmissions2['cldclocount'] = nysubmissions2['CASENBR'].map(cldclocounts).fillna(0).astype(int)

# Add a column to count the total number of "Closed Granted" and "Accepted" submissions for each CASENBR
clgacccount = nysubmissions2[nysubmissions2['CDSTATUS'].isin(['CLG', 'ACC'])].groupby('CASENBR').size()
nysubmissions2['clgacccount'] = nysubmissions2['CASENBR'].map(clgacccount).fillna(0).astype(int)

# Make sure 'sitdtrcv' and 'sdtstat' are in datetime format
nysubmissions2['sitdtrcv'] = pd.to_datetime(nysubmissions2['sitdtrcv'])
nysubmissions2['sdtstat'] = pd.to_datetime(nysubmissions2['sdtstat'])

# Find the earliest sitdtrcv date for each CASENBR
earliestsitdtrcv = nysubmissions2.groupby('CASENBR')['sitdtrcv'].min().reset_index()
earliestsitdtrcv.rename(columns={'sitdtrcv': 'earliest_sitdtrcv'}, inplace=True)

# Find the latest sdtstat date for each CASENBR
latestsdtstat = nysubmissions2.groupby('CASENBR')['sdtstat'].max().reset_index()
latestsdtstat.rename(columns={'sdtstat': 'latest_sdtstat'}, inplace=True)

# Merge the earliest submission date and most recent status update date
merged_dates = pd.merge(earliestsitdtrcv, latestsdtstat, on='CASENBR')

# Calculate the number of days between the earliest submission date and the most recent status update date
merged_dates['days_between'] = (merged_dates['latest_sdtstat'] - merged_dates['earliest_sitdtrcv']).dt.days

# Merge the calculated dates back into the original dataset
nysubmissionsenriched = pd.merge(
    nysubmissions2,
    merged_dates,
    on='CASENBR',
    how='left'
)

# save the updated dataset as NYSubmissionsEnriched.csv
nysubmissionsenriched.to_csv("../results/data/NY_SubmissionsEnriched.csv", index=False)

In [25]:
# STEP 3: Create expanded dataset of NYSubmissionsEnriched.csv with codes translated for easier use 

# Import code translation CSVs
complaintcodes = pd.read_csv(r'../data/cdsub1cb_ConcatSubjectCodes.csv')
facilitycodes = pd.read_csv(r'../data/CDFC_FacilityCodes.csv')
statuscodes = pd.read_csv(r'../data/CDSTATUS_CaseStatusCodes.csv')
orglevelcodes = pd.read_csv(r'../data/ITERLVL_OrgLevelCodes.csv')
statusreasoncodes = pd.read_csv(r'../data/STATRSN_StatusReasonCodes.csv')
columncodes = pd.read_csv(r'../data/ColumnCodes.csv')
primarysubjectcodes = pd.read_csv(r'../data/CDSUB1PR _PrimarySubjectCodes.csv')

# Load NY submission dataset
nysubmissionsrenriched2 = nysubmissionsenriched.copy()

# Duplicate the cdsub1cb column to create cdsub1cbTEXT column (for translation of primary and secondary subject)
nysubmissionsrenriched2['cdsub1cbTEXT'] = nysubmissionsrenriched2['cdsub1cb']

# Find and replace code values with translations from CSVs
nysubmissionsrenriched2['cdsub1cbTEXT'] = nysubmissionsrenriched2['cdsub1cbTEXT'].map(complaintcodes.set_index('Code')['Text']).fillna(nysubmissionsrenriched2['cdsub1cbTEXT'])
nysubmissionsrenriched2['CDFCLEVN'] = nysubmissionsrenriched2['CDFCLEVN'].map(facilitycodes.set_index('Facility_Code')['Facility_Name']).fillna(nysubmissionsrenriched2['CDFCLEVN'])
nysubmissionsrenriched2['CDFCLRCV'] = nysubmissionsrenriched2['CDFCLRCV'].map(facilitycodes.set_index('Facility_Code')['Facility_Name']).fillna(nysubmissionsrenriched2['CDFCLRCV'])
nysubmissionsrenriched2['CDOFCRCV'] = nysubmissionsrenriched2['CDOFCRCV'].map(facilitycodes.set_index('Facility_Code')['Facility_Name']).fillna(nysubmissionsrenriched2['CDOFCRCV'])
nysubmissionsrenriched2['ITERLVL'] = nysubmissionsrenriched2['ITERLVL'].map(orglevelcodes.set_index('Code')['Text']).fillna(nysubmissionsrenriched2['ITERLVL'])
nysubmissionsrenriched2['CDSTATUS'] = nysubmissionsrenriched2['CDSTATUS'].map(statuscodes.set_index('Code')['Text']).fillna(nysubmissionsrenriched2['CDSTATUS'])
nysubmissionsrenriched2['STATRSN1'] = nysubmissionsrenriched2['STATRSN1'].map(statusreasoncodes.set_index('Reason Code')['Text']).fillna(nysubmissionsrenriched2['STATRSN1'])
nysubmissionsrenriched2['STATRSN2'] = nysubmissionsrenriched2['STATRSN2'].map(statusreasoncodes.set_index('Reason Code')['Text']).fillna(nysubmissionsrenriched2['STATRSN2'])
nysubmissionsrenriched2['STATRSN3'] = nysubmissionsrenriched2['STATRSN3'].map(statusreasoncodes.set_index('Reason Code')['Text']).fillna(nysubmissionsrenriched2['STATRSN3'])
nysubmissionsrenriched2['STATRSN4'] = nysubmissionsrenriched2['STATRSN4'].map(statusreasoncodes.set_index('Reason Code')['Text']).fillna(nysubmissionsrenriched2['STATRSN4'])
nysubmissionsrenriched2['STATRSN5'] = nysubmissionsrenriched2['STATRSN5'].map(statusreasoncodes.set_index('Reason Code')['Text']).fillna(nysubmissionsrenriched2['STATRSN5'])
nysubmissionsrenriched2['CDSUB1PR'] = nysubmissionsrenriched2['CDSUB1PR'].map(primarysubjectcodes.set_index('Primary Subject Code')['Primary Subject Code Translation']).fillna(nysubmissionsrenriched2['CDSUB1PR'])

# If status is "rejected," there will not be a value in the Remedy Case Submission due date. Replace null with "Rejected" to make reason clear
nysubmissionsrenriched2['sdtdue'] = nysubmissionsrenriched2['sdtdue'].fillna('rejected')

# Replace binary values with "yes" and "no" for clarity
nysubmissionsrenriched2['accept'] = nysubmissionsrenriched2['accept'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['reject'] = nysubmissionsrenriched2['reject'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['deny'] = nysubmissionsrenriched2['deny'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['grant'] = nysubmissionsrenriched2['grant'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['other'] = nysubmissionsrenriched2['other'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['submit'] = nysubmissionsrenriched2['submit'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['filed'] = nysubmissionsrenriched2['filed'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['closed'] = nysubmissionsrenriched2['closed'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['diffreg_filed'] = nysubmissionsrenriched2['diffreg_filed'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['diffinst'] = nysubmissionsrenriched2['diffinst'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['timely'] = nysubmissionsrenriched2['timely'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['diffreg_answer'] = nysubmissionsrenriched2['diffreg_answer'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['overdue'] = nysubmissionsrenriched2['overdue'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['untimely'] = nysubmissionsrenriched2['untimely'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['resubmit'] = nysubmissionsrenriched2['resubmit'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['noinfres'] = nysubmissionsrenriched2['noinfres'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['attachmt'] = nysubmissionsrenriched2['attachmt'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['wronglvl'] = nysubmissionsrenriched2['wronglvl'].replace({0: 'no', 1: 'yes'})
nysubmissionsrenriched2['otherrej'] = nysubmissionsrenriched2['otherrej'].replace({0: 'no', 1: 'yes'})

# Load column code CSV as a dictionary
columncodes_dict = dict(zip(columncodes['Code'], columncodes['Text']))

# Replace coloumn header codes with translations
nysubmissionsrenriched2.rename(columns=columncodes_dict, inplace=True)

# Save enriched NY dataset to a CSV
nysubmissionsrenriched2.to_csv('../results/data/NY_SubmissionsEnrichedExpanded.csv', index=False)

In [26]:
# STEP 4: Create subset of NY submissions data which only has unique complaints. The one record/row with the most recent status update from each Case Number will be present in this dataset.

# Load enriched NY dataset 
nysubmissionedenriched3 = pd.read_csv(r'../results/data/NY_SubmissionsEnriched.csv')

# Sort "sitdtrcv" (submission date) in descending order
nysubmissionedenriched3 = nysubmissionedenriched3.sort_values(by='sitdtrcv', ascending=False)

# Drop duplicate CASENBRs, keeping the most recent (highest sitdtrcv)
unique_nysubmissionedenriched1 = nysubmissionedenriched3.drop_duplicates(subset='CASENBR', keep='first')

# Save Unique NY Submissions to a CSV
unique_nysubmissionedenriched1.to_csv("../results/data/NY_UniqueComplaintsEnriched.csv", index=False)

In [27]:
# STEP 5: Create expanded dataset of NYSubmissionsEnriched.csv with codes translated for easier use 

# Code translation CSVs have already been loaded above in STEP 3

# Save Unique NY Submissions as another variable (to not mess with the original variable)
unique_nysubmissionsrenriched2 = unique_nysubmissionedenriched1.copy()

# Duplicate the cdsub1cb column to create cdsub1cbTEXT column (for translation of primary and secondary subject)
unique_nysubmissionsrenriched2.loc[:, 'cdsub1cbTEXT'] = unique_nysubmissionsrenriched2['cdsub1cb']

# Find and replace code values with translations from CSVs
unique_nysubmissionsrenriched2['cdsub1cbTEXT'] = unique_nysubmissionsrenriched2['cdsub1cbTEXT'].map(complaintcodes.set_index('Code')['Text']).fillna(unique_nysubmissionsrenriched2['cdsub1cbTEXT'])
unique_nysubmissionsrenriched2['CDFCLEVN'] = unique_nysubmissionsrenriched2['CDFCLEVN'].map(facilitycodes.set_index('Facility_Code')['Facility_Name']).fillna(unique_nysubmissionsrenriched2['CDFCLEVN'])
unique_nysubmissionsrenriched2['CDFCLRCV'] = unique_nysubmissionsrenriched2['CDFCLRCV'].map(facilitycodes.set_index('Facility_Code')['Facility_Name']).fillna(unique_nysubmissionsrenriched2['CDFCLRCV'])
unique_nysubmissionsrenriched2['CDOFCRCV'] = unique_nysubmissionsrenriched2['CDOFCRCV'].map(facilitycodes.set_index('Facility_Code')['Facility_Name']).fillna(unique_nysubmissionsrenriched2['CDOFCRCV'])
unique_nysubmissionsrenriched2['ITERLVL'] = unique_nysubmissionsrenriched2['ITERLVL'].map(orglevelcodes.set_index('Code')['Text']).fillna(unique_nysubmissionsrenriched2['ITERLVL'])
unique_nysubmissionsrenriched2['CDSTATUS'] = unique_nysubmissionsrenriched2['CDSTATUS'].map(statuscodes.set_index('Code')['Text']).fillna(unique_nysubmissionsrenriched2['CDSTATUS'])
unique_nysubmissionsrenriched2['STATRSN1'] = unique_nysubmissionsrenriched2['STATRSN1'].map(statusreasoncodes.set_index('Reason Code')['Text']).fillna(unique_nysubmissionsrenriched2['STATRSN1'])
unique_nysubmissionsrenriched2['STATRSN2'] = unique_nysubmissionsrenriched2['STATRSN2'].map(statusreasoncodes.set_index('Reason Code')['Text']).fillna(unique_nysubmissionsrenriched2['STATRSN2'])
unique_nysubmissionsrenriched2['STATRSN3'] = unique_nysubmissionsrenriched2['STATRSN3'].map(statusreasoncodes.set_index('Reason Code')['Text']).fillna(unique_nysubmissionsrenriched2['STATRSN3'])
unique_nysubmissionsrenriched2['STATRSN4'] = unique_nysubmissionsrenriched2['STATRSN4'].map(statusreasoncodes.set_index('Reason Code')['Text']).fillna(unique_nysubmissionsrenriched2['STATRSN4'])
unique_nysubmissionsrenriched2['STATRSN5'] = unique_nysubmissionsrenriched2['STATRSN5'].map(statusreasoncodes.set_index('Reason Code')['Text']).fillna(unique_nysubmissionsrenriched2['STATRSN5'])
unique_nysubmissionsrenriched2['CDSUB1PR'] = unique_nysubmissionsrenriched2['CDSUB1PR'].map(primarysubjectcodes.set_index('Primary Subject Code')['Primary Subject Code Translation']).fillna(unique_nysubmissionsrenriched2['CDSUB1PR'])

# If status is "rejected," there will not be a value in the Remedy Case Submission due date. Replace null with "Rejected" to make reason clear
unique_nysubmissionsrenriched2['sdtdue'] = unique_nysubmissionsrenriched2['sdtdue'].fillna('rejected')

# Replace binary values with "yes" and "no" for clarity
unique_nysubmissionsrenriched2['accept'] = unique_nysubmissionsrenriched2['accept'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['reject'] = unique_nysubmissionsrenriched2['reject'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['deny'] = unique_nysubmissionsrenriched2['deny'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['grant'] = unique_nysubmissionsrenriched2['grant'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['other'] = unique_nysubmissionsrenriched2['other'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['submit'] = unique_nysubmissionsrenriched2['submit'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['filed'] = unique_nysubmissionsrenriched2['filed'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['closed'] = unique_nysubmissionsrenriched2['closed'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['diffreg_filed'] = unique_nysubmissionsrenriched2['diffreg_filed'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['diffinst'] = unique_nysubmissionsrenriched2['diffinst'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['timely'] = unique_nysubmissionsrenriched2['timely'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['diffreg_answer'] = unique_nysubmissionsrenriched2['diffreg_answer'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['overdue'] = unique_nysubmissionsrenriched2['overdue'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['untimely'] = unique_nysubmissionsrenriched2['untimely'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['resubmit'] = unique_nysubmissionsrenriched2['resubmit'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['noinfres'] = unique_nysubmissionsrenriched2['noinfres'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['attachmt'] = unique_nysubmissionsrenriched2['attachmt'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['wronglvl'] = unique_nysubmissionsrenriched2['wronglvl'].replace({0: 'no', 1: 'yes'})
unique_nysubmissionsrenriched2['otherrej'] = unique_nysubmissionsrenriched2['otherrej'].replace({0: 'no', 1: 'yes'})

# column code CSV has already been loaded as a dictionary
# Replace coloumn header codes with translations
unique_nysubmissionsrenriched2.rename(columns=columncodes_dict, inplace=True)

# Save enriched unique NY dataset to a CSV
unique_nysubmissionsrenriched2.to_csv('../results/data/NY_UniqueComplaintsEnrichedExpanded.csv', index=False)

# test 